<a href="https://colab.research.google.com/github/CodesHassan/Crop_Yield_Prediction/blob/main/Crop_Yield_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import pandas as pd

data=pd.read_csv("/content/crop_yield_data.csv")
data.head(3)

,rainfall_mm,soil_quality_index,farm_size_hectares,sunlight_hours,fertilizer_kg,crop_yield
0,1626,9,636,11,1006,404
1,1959,9,73,11,112,115
2,1360,1,352,5,702,231


In [76]:
data.isnull().sum()

,0
rainfall_mm,0
soil_quality_index,0
farm_size_hectares,0
sunlight_hours,0
fertilizer_kg,0
crop_yield,0


In [77]:
data.describe()

,rainfall_mm,soil_quality_index,farm_size_hectares,sunlight_hours,fertilizer_kg,crop_yield
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,1263.095000,5.506667,498.801000,7.995333,1549.450333,328.099000
std,432.371756,2.855172,287.122742,2.621501,814.326919,145.036503
min,500.000000,1.000000,10.000000,4.000000,100.000000,46.000000
25%,896.000000,3.000000,242.000000,6.000000,869.750000,199.000000
50%,1277.000000,6.000000,505.000000,8.000000,1542.000000,332.000000
75%,1636.000000,8.000000,741.000000,10.000000,2225.000000,455.000000
max,2000.000000,10.000000,1000.000000,12.000000,3000.000000,628.000000


In [78]:
# Drop unnecessary columns and separate target variable
X = data.drop(columns=["crop_yield"])  # Correct way to drop a column
y = data['crop_yield']  # Target variable

In [79]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Standard scaling the features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Display scaled features
X_scaled

,rainfall_mm,soil_quality_index,farm_size_hectares,sunlight_hours,fertilizer_kg
0,0.839476,1.223715,0.477921,1.146354,-0.667473
1,1.609774,1.223715,-1.483240,1.146354,-1.765495
2,0.224162,-1.578685,-0.511368,-1.142793,-1.040849
3,1.228095,-1.228385,1.564745,-0.379744,-1.535819
4,0.848728,-0.177485,1.341807,-1.142793,1.453651
...,...,...,...,...,...
2995,0.508686,1.223715,-1.465823,1.527878,1.461020
2996,-1.061983,1.223715,-0.062008,-1.524317,0.428093
2997,-0.909311,-1.578685,-1.343904,-0.379744,0.899727
2998,0.205656,0.172815,0.934248,1.146354,1.659991


In [80]:
from sklearn.model_selection import train_test_split,GridSearchCV
X_train,  X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [81]:
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
import numpy as np

# Define the Gradient Boosting model with the best hyperparameters
best_gb_model = GradientBoostingRegressor(
    learning_rate=0.05,
    max_depth=7,
    n_estimators=100,
    subsample=0.8
)

# Add Bagging to Gradient Boosting
bagging_gb_model = BaggingRegressor(estimator=best_gb_model, n_estimators=10)

# Train the Bagging + Gradient Boosting model
bagging_gb_model.fit(X_train, y_train)

# Test score on train and test sets
print(f"Bagging with Gradient Boosting Train Score: {bagging_gb_model.score(X_train, y_train) * 100}")
print(f"Bagging with Gradient Boosting Test Score: {bagging_gb_model.score(X_test, y_test) * 100}")

y_pred_gb = bagging_gb_model.predict(X_test)
mse_gb = mean_squared_error(y_test, y_pred_gb)
rmse_gb = np.sqrt(mse_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print("\n--- Gradient Boosting ---")
print("RMSE:", rmse_gb)
print("R^2 :", r2_gb)

Bagging with Gradient Boosting Train Score: 99.96927514016024
Bagging with Gradient Boosting Test Score: 99.90243904482027

--- Gradient Boosting ---
RMSE: 4.570826258707254
R^2 : 0.9990243904482027


In [82]:
new_data=pd.DataFrame({
    "Original_pred": y_test,
    "Prediction": y_pred_gb
})

new_data

,Original_pred,Prediction
1801,181,176.369799
1190,309,307.752952
1817,291,295.896888
251,172,181.745951
2505,261,263.259510
...,...,...
104,191,193.627682
2087,398,400.096272
599,301,296.965033
1756,257,256.534311
